In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db


# Fetch the service account key JSON file contents
cred = credentials.Certificate('/Users/Solomon/Desktop/cauNotiPrivate/cau-hashkeyword-serviceAccountKey.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://cau-hashkeyword.firebaseio.com'
})

In [171]:
class Info_webpage:
    def setData(self, title, date, url):
        self.title = title
        self.date = date
        self.url = url
    def getData(self):
        self.dataSet = []
        for i in range(len(self.title)):
            self.dataSet.append((self.title[i], self.date[i], self.url[i]))
        return self.dataSet
    def getTitle(self):
        return self.title
    def getDate(self):
        return self.date
    def getUrl(self):
        return self.url

In [172]:
class Info_user:
    def __init__(self):
        self.match = []
    def setData(self, user, keywords, selectedWebsite, fcmToken): # 나중에 uid 부분은 빼도 무방
        self.uid = user
        self.keywords = keywords
        self.selectedWebsite = selectedWebsite
        self.fcmToken = fcmToken
    def setSelectedWebsite(self, selectedWebsite): # parsing한 웹페이지 이름 저장
        self.selectedWebsite = selectedWebsite
    def getUid(self):
        return self.uid
    def getKeywords(self):
        return self.keywords
    def getSelectedWebsite(self):
        return self.selectedWebsite
    def getFcmToken(self):
        return self.fcmToken
    def setPushData(self, matchKeyword, matchTitle, matchDate, matchURL):
        self.match.append((matchKeyword, matchTitle, matchDate, matchURL))
    def getPushData(self):
        return self.match
    def initPushData(self): # Push 알림을 보낸 후 초기화 과정
        self.match = []

In [385]:
ref = db.reference('crawling/webpages')

cau = Info_webpage()
lib = Info_webpage()
dorm = Info_webpage()
ict = Info_webpage()
cse = Info_webpage()

webpages = {'cau': cau, 'library': lib, 'dormitory': dorm, 'ict': ict , 'cse': cse}

for (name, page) in webpages.items():
    page.setData(ref.child(name).child('title').get(),
                 ref.child(name).child('date').get(),
                 ref.child(name).child('url').get())

In [386]:
ref = db.reference('users')

users = ref.get() # 유저의 uid 가져옴
# users_uid = ref.get().keys()
users_info = []

for user in users:
    infoUser = Info_user() # 객체를 유저 하나씩 만듬
    infoUser.setData(user,
                     ref.child(user).child('keywords').get(),
                     ref.child(user).child('selectedWebsite').get(),
                     ref.child(user).child('fcmToken').get())
    user_parseSelectedWebsite = []
    for selected in infoUser.getSelectedWebsite():
        user_parseSelectedWebsite.append(selected.split('(')[1][:-1].split('.')[0])
    infoUser.setSelectedWebsite(user_parseSelectedWebsite)
    users_info.append(infoUser)

In [387]:
# 4중 for문 최적화 요망 (추후 업데이트) - 물론 크롤링 주기를 짧게하면 title을 도는 부분이 적게 걸림
# 유저를 순차적으로 선택하며 해당 유저가 선택한 웹사이트에서 키워드를 공지사항의 제목과 비교

for user in users_info:
    for site in user.getSelectedWebsite():
        for keyword in user.getKeywords():
            for (title, date, url) in webpages[site].getData():
                if keyword in title:
                    user.setPushData(keyword, title, date, url)

In [388]:
# ios Application에 push notification 전송

In [389]:
# user별로 push할 데이터를 담아둔 match 리스트 초기화 (match는 이번 실행에 보낼 데이터만을 저장해 둠)
# for user in users_info:
#    user.initPushData()

In [390]:
fcmTokens = {}
pushData = {}

for user in users_info:
    if(len(user.getPushData()) > 0):
        # print(user.getUid())
        # print(user.getFcmToken())
        print(user.getPushData())
        print()
        fcmTokens[user.getFcmToken()] = user.getPushData()
        pushData[user.getUid()] = user.getPushData()
        # fcmTokens와 pushData는 fcmToken 외에는 데이터의 차이가 없지만, 일단 기능에 따라 분리하였다.
        
# fcmTokens # 같은 기기에 아이디를 2개 등록하면 하나의 fcmToken에 덮어 쓰여진다.
# pushData

[('입관', '[생활관] 생활관 입관 가능시간 안내/Dormitory Admission Time Information', '2016.06.23', 'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=1611&bbsID=notice'), ('막역지룸', '[막역지룸] 2019학년도 1학기 "막역지룸(莫逆之Room)" 행사 안내/“Giving Compliments to Your Roommate” Program Information', '2019.04.03', 'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=2769&bbsID=notice'), ('막역지룸', '[막역지룸] 2019-1학기 막역지룸 선발 공지/Roommate program notice', '2019.04.17', 'https://dormitory.cau.ac.kr/bbs/bbs_view.php?pNum=2780&bbsID=notice'), ('첨단영상대학원', '2019학년도 후반기 첨단영상대학원 신입생 모집 ', '2019.04.17', 'http://ict.cau.ac.kr/20150610/sub05/sub05_01_list.php?cmd=view&cpage=1&idx=1300&search_gbn=1&search_keyword=')]



In [391]:
# https://github.com/olucurious/PyFCM
from pyfcm import FCMNotification
import json

with open("/Users/Solomon/Desktop/fcm-server-key.json") as json_file:
    json_data = json.load(json_file)
    server_key = json_data["server_key"]

push_service = FCMNotification(api_key=server_key)

# OR initialize with proxies
# proxy_dict = {
#           "http"  : "http://127.0.0.1",
#           "https" : "http://127.0.0.1",
#         }
# push_service = FCMNotification(api_key="<api-key>", proxy_dict=proxy_dict)

# Your api-key can be gotten from:  https://console.firebase.google.com/project/<project-name>/settings/cloudmessaging

# 알림 메시지
for (token, data) in fcmTokens.items():
    registration_id = token
    for i in range(len(data)):
        # 공지사항의 출처를 찾는다. (URL주소에서)
        reference = data[i][3].split("//")[1].split(".")[0]
        if reference == "cau":
            reference = "CAU_NOTICE"
        elif reference == "library":
            reference = "도서관"
        elif reference == "dormitory":
            reference = "생활관"
        elif reference == "ict":
            reference = "창의ICT공과대학"
        elif refference == "cse":
            reference = "소프트웨어학부"
        else: # 만약 해당되는 것이 없다면
            reference = ""
    
        message_title = "중앙대학교(서울) " + reference + "   #" + data[i][0]
        message_body = data[i][1]
        result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)

    print(result)
    

# 앱이 TimelineView를 실행했을 때 전달되어야 하는 정보들을 DB에 올려둠
ref = db.reference('users')
for user in users:
    if ref.child(user).child('fcmToken').get() in fcmTokens.keys():
        ref.child(user).child('pushData').set(pushData[user])
        # 어차피 매번 갱신되어야 하므로.. 이전 데이터는 필요 x (이전알림들은 각 디바이스에 아카이빙된다.)        
        
# 각 pushData를 구분하지 않고 firebase DB에 순서대로 저장함     
#         mergePushData = []
#         for data in pushData[user]:
#             mergePushData.append(data[0])
#             mergePushData.append(data[1])
#             mergePushData.append(data[2])
#             mergePushData.append(data[3])
#         ref.child(user).child('pushData').set(mergePushData)

    

# 데이터 메시지로 정보를 보내주는 것이 아닌 firebase에 알림 메시지를 준 내용으로 pushData 항목을 만들기
# 그리고 앱을 실행하여 타임라인 뷰에 들어가면 pushData 항목과 테이블 뷰를 동기화

# 데이터 메시지
# i = 0
# for (token, data) in fcmTokens.items():
#     registration_id = token
#     data_message = {
#         "keyword": data[i][0],
#         "title": data[i][1],
#         "date": data[i][2],
#         "url": data[i][3]
#     }
#     result = push_service.single_device_data_message(registration_id=registration_id, data_message=data_message)
#     i += 1
    
#     print(result)
    
# 알림메시지와 데이터메시지를 같이 보내면 사용자가 알림을 탭(오픈)한 경우에만, 앱이 데이터 페이로드를 처리하므로 나눠서 작업하자.
# 출처: https://pliss.tistory.com/116

{'multicast_ids': [4988082565484093075], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [{'message_id': '1561464312986830'}], 'topic_message_id': None}
